# Portfolio Predictive Modeling

This notebook demonstrates end-to-end predictive modeling for portfolio price direction prediction using AAPL and MSFT stocks.

## Objectives
1. Import and initialize the PredictiveModelAgent
2. Train a binary classification model on AAPL and MSFT features
3. Evaluate model performance with comprehensive metrics
4. Register the trained model in Unity Catalog
5. Display performance visualizations and insights

## Prerequisites
- Feature engineering must be completed (run `03_feature_engineering.ipynb`)
- Unity Catalog must be configured with appropriate permissions
- MLflow tracking must be enabled in the Databricks workspace

In [0]:
# Import required libraries
import sys
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Add src to path for imports
sys.path.append('../src')

# Import our custom agent
from agents.predictive_model_agent import PredictiveModelAgent, PredictiveModelError

print("Libraries imported successfully!")
print(f"Notebook executed at: {datetime.now()}")

## Configuration and Setup

Configure the predictive modeling environment for our specific use case.

In [0]:
# Configuration
CATALOG_NAME = "finance_catalog"
SCHEMA_NAME = "silver"
TARGET_TICKERS = ['AAPL', 'MSFT']
MODEL_NAME = "portfolio_price_predictor_aapl_msft"

print("=" * 60)
print("PREDICTIVE MODELING CONFIGURATION")
print("=" * 60)
print(f"Unity Catalog: {CATALOG_NAME}")
print(f"Schema: {SCHEMA_NAME}")
print(f"Target Tickers: {', '.join(TARGET_TICKERS)}")
print(f"Model Name: {MODEL_NAME}")
print("=" * 60)

## Initialize Predictive Model Agent

Create and configure the PredictiveModelAgent with our Unity Catalog settings.

In [0]:
# Initialize the Predictive Model Agent
try:
    agent = PredictiveModelAgent(
        catalog=CATALOG_NAME,
        schema=SCHEMA_NAME
    )
    
    print("✅ PredictiveModelAgent initialized successfully!")
    print(f"📊 Spark Session: {type(agent.spark).__name__}")
    print(f"📈 MLflow Client: {type(agent.mlflow_client).__name__ if agent.mlflow_client else 'Not initialized'}")
    print(f"🎯 Feature Columns: {agent.feature_cols}")
    print(f"🏷️  Label Column: {agent.label_col}")
    
except Exception as e:
    print(f"❌ Failed to initialize PredictiveModelAgent: {str(e)}")
    print("Please ensure Spark is properly configured and accessible.")
    raise

## Data Verification

Before training, let's verify that the required feature tables exist and contain data.

In [0]:
# Verify feature tables exist
print("🔍 Verifying feature tables...")
print()

table_status = {}
for ticker in TARGET_TICKERS:
    table_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.features_{ticker}"
    try:
        exists = agent.spark.catalog.tableExists(table_name)
        table_status[ticker] = exists
        
        if exists:
            # Get row count
            row_count = agent.spark.table(table_name).count()
            print(f"✅ {ticker}: Table exists with {row_count:,} rows")
        else:
            print(f"❌ {ticker}: Table {table_name} does not exist")
            
    except Exception as e:
        print(f"⚠️  {ticker}: Error checking table - {str(e)}")
        table_status[ticker] = False

# Check if we can proceed
available_tickers = [ticker for ticker, exists in table_status.items() if exists]
print(f"\n📋 Available tickers for training: {available_tickers}")

if len(available_tickers) == 0:
    print("⚠️  No feature tables available. Please run feature engineering first.")
elif len(available_tickers) < len(TARGET_TICKERS):
    print(f"⚠️  Only {len(available_tickers)}/{len(TARGET_TICKERS)} tables available.")
    print("Proceeding with available tickers.")
    TARGET_TICKERS = available_tickers
else:
    print("✅ All required feature tables are available!")

## Model Training

Train the predictive model using the available feature data.

In [0]:
# Train the predictive model
if len(TARGET_TICKERS) > 0:
    print(f"🚀 Starting model training for: {', '.join(TARGET_TICKERS)}")
    print(f"⏰ Training started at: {datetime.now()}")
    print()
    
    try:
        # Train with hyperparameter tuning for better performance
        training_results = agent.train(
            tickers=TARGET_TICKERS,
            hyperparameter_tuning=True
        )
        
        print("🎉 Model training completed successfully!")
        print(f"⏰ Training finished at: {datetime.now()}")
        print()
        
        # Store results for later use
        model = training_results['model']
        metrics = training_results['metrics']
        feature_importance = training_results.get('feature_importance', [])
        
        print("📊 Training Results Summary:")
        print("-" * 40)
        for metric_name, value in metrics.items():
            print(f"{metric_name:20}: {value:.4f}")
        
    except PredictiveModelError as e:
        print(f"❌ Training failed with PredictiveModelError: {str(e)}")
        model = None
        metrics = None
        
    except Exception as e:
        print(f"❌ Training failed with unexpected error: {str(e)}")
        model = None
        metrics = None
        
else:
    print("⚠️  No tickers available for training. Skipping model training.")
    model = None
    metrics = None

## Performance Evaluation

Detailed analysis of model performance with visualizations.

In [0]:
# Performance evaluation and visualization
if metrics is not None:
    print("📈 DETAILED PERFORMANCE ANALYSIS")
    print("=" * 50)
    
    # Create performance visualization
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Training vs Test Performance Comparison
    train_metrics = ['train_auc', 'train_accuracy', 'train_f1']
    test_metrics = ['test_auc', 'test_accuracy', 'test_f1']
    
    train_values = [metrics.get(m, 0) for m in train_metrics]
    test_values = [metrics.get(m, 0) for m in test_metrics]
    metric_names = ['AUC', 'Accuracy', 'F1-Score']
    
    x = range(len(metric_names))
    width = 0.35
    
    ax1.bar([i - width/2 for i in x], train_values, width, label='Training', alpha=0.8)
    ax1.bar([i + width/2 for i in x], test_values, width, label='Test', alpha=0.8)
    ax1.set_xlabel('Metrics')
    ax1.set_ylabel('Score')
    ax1.set_title('Training vs Test Performance')
    ax1.set_xticks(x)
    ax1.set_xticklabels(metric_names)
    ax1.legend()
    ax1.set_ylim(0, 1)
    
    # 2. All Metrics Overview
    all_metrics = list(metrics.keys())
    all_values = list(metrics.values())
    
    colors = ['skyblue' if 'train' in m else 'lightcoral' for m in all_metrics]
    bars = ax2.barh(all_metrics, all_values, color=colors, alpha=0.7)
    ax2.set_xlabel('Score')
    ax2.set_title('All Performance Metrics')
    ax2.set_xlim(0, 1)
    
    # Add value labels on bars
    for bar, value in zip(bars, all_values):
        ax2.text(value + 0.01, bar.get_y() + bar.get_height()/2, 
                f'{value:.3f}', va='center', fontsize=8)
    
    # 3. Feature Importance (if available)
    if feature_importance and len(feature_importance) > 0:
        features = [f['feature'] for f in feature_importance[:8]]  # Top 8 features
        importances = [f['importance'] for f in feature_importance[:8]]
        
        ax3.barh(features, importances, color='lightgreen', alpha=0.7)
        ax3.set_xlabel('Importance')
        ax3.set_title('Top Feature Importances')
        
        # Add value labels
        for i, (feature, importance) in enumerate(zip(features, importances)):
            ax3.text(importance + 0.001, i, f'{importance:.3f}', 
                    va='center', fontsize=8)
    else:
        ax3.text(0.5, 0.5, 'Feature importance\nnot available', 
                ha='center', va='center', transform=ax3.transAxes)
        ax3.set_title('Feature Importance (N/A)')
    
    # 4. Model Quality Assessment
    quality_metrics = {
        'Overfitting Risk': abs(metrics.get('train_auc', 0) - metrics.get('test_auc', 0)),
        'Prediction Quality': metrics.get('test_auc', 0),
        'Accuracy': metrics.get('test_accuracy', 0),
        'Precision': metrics.get('test_precision', 0)
    }
    
    # Create radar-like visualization
    quality_names = list(quality_metrics.keys())
    quality_values = list(quality_metrics.values())
    
    # Invert overfitting risk (lower is better)
    quality_values[0] = 1 - min(quality_values[0], 1.0)
    
    ax4.pie(quality_values, labels=quality_names, autopct='%1.3f', startangle=90)
    ax4.set_title('Model Quality Assessment')
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed metrics
    print("\n📊 DETAILED METRICS:")
    print("-" * 30)
    for metric, value in metrics.items():
        category = "Training" if "train" in metric else "Test"
        metric_clean = metric.replace("train_", "").replace("test_", "").upper()
        print(f"{category:10} {metric_clean:10}: {value:.4f}")
    
    # Model quality assessment
    test_auc = metrics.get('test_auc', 0)
    overfitting = abs(metrics.get('train_auc', 0) - test_auc)
    
    print("\n🎯 MODEL QUALITY ASSESSMENT:")
    print("-" * 30)
    if test_auc >= 0.8:
        print("✅ Excellent prediction quality (AUC ≥ 0.8)")
    elif test_auc >= 0.7:
        print("✅ Good prediction quality (AUC ≥ 0.7)")
    elif test_auc >= 0.6:
        print("⚠️  Moderate prediction quality (AUC ≥ 0.6)")
    else:
        print("❌ Poor prediction quality (AUC < 0.6)")
    
    if overfitting <= 0.05:
        print("✅ Low overfitting risk (< 5% gap)")
    elif overfitting <= 0.1:
        print("⚠️  Moderate overfitting risk (5-10% gap)")
    else:
        print("❌ High overfitting risk (> 10% gap)")
        
else:
    print("⚠️  No metrics available to display.")

## Model Registration

Register the trained model in Unity Catalog for production use.

In [0]:
# Register the model in Unity Catalog
if model is not None and metrics is not None:
    print("📝 Registering model in Unity Catalog...")
    
    try:
        # Create comprehensive model description
        model_description = f"""
        Portfolio Price Direction Predictor
        
        Training Details:
        - Tickers: {', '.join(TARGET_TICKERS)}
        - Algorithm: Gradient Boosted Trees (GBT)
        - Features: {', '.join(agent.feature_cols)}
        - Training Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
        
        Performance Metrics:
        - Test AUC: {metrics.get('test_auc', 0):.4f}
        - Test Accuracy: {metrics.get('test_accuracy', 0):.4f}
        - Test F1-Score: {metrics.get('test_f1', 0):.4f}
        
        Model predicts next-day price direction (up=1, down=0) for portfolio optimization.
        """.strip()
        
        registration_info = agent.register_model(
            model_name=MODEL_NAME,
            description=model_description
        )
        
        print("🎉 Model registered successfully!")
        print()
        print("📋 REGISTRATION DETAILS:")
        print("-" * 40)
        print(f"Model Name: {registration_info['model_name']}")
        print(f"Version: {registration_info['model_version']}")
        print(f"URI: {registration_info['model_uri']}")
        print(f"Registered: {registration_info['registration_time']}")
        
        # Success summary
        print("\n" + "="*60)
        print("🎊 PREDICTIVE MODELING COMPLETED SUCCESSFULLY!")
        print("="*60)
        print(f"✅ Model trained on {len(TARGET_TICKERS)} tickers: {', '.join(TARGET_TICKERS)}")
        print(f"✅ Performance: AUC = {metrics.get('test_auc', 0):.4f}, Accuracy = {metrics.get('test_accuracy', 0):.4f}")
        print(f"✅ Model registered: {MODEL_NAME} v{registration_info['model_version']}")
        print(f"✅ Ready for production deployment!")
        
    except Exception as e:
        print(f"❌ Model registration failed: {str(e)}")
        print("Model training was successful but registration encountered issues.")
        print("Please check Unity Catalog permissions and MLflow configuration.")
        
else:
    print("⚠️  No model available for registration.")
    print("Please ensure model training completed successfully.")

## Next Steps and Production Deployment

Guidelines for using the trained model in production environments.

In [0]:
# Display next steps and production guidance
print("🚀 NEXT STEPS FOR PRODUCTION DEPLOYMENT")
print("=" * 50)
print()

if model is not None:
    print("✅ Model Training: COMPLETED")
    print("✅ Performance Evaluation: COMPLETED")
    print("✅ Model Registration: COMPLETED")
    print()
    print("📋 PRODUCTION CHECKLIST:")
    print("1. 🔍 Review model performance metrics above")
    print("2. 🧪 Test model predictions on recent data")
    print("3. 🔒 Set up model monitoring and alerts")
    print("4. 📊 Configure automated retraining schedule")
    print("5. 🚀 Deploy model to production serving infrastructure")
    print()
    print("💡 MODEL USAGE EXAMPLE:")
    print(f"   model_uri = '{registration_info.get('model_uri', 'models:/' + MODEL_NAME + '/latest')}'")
    print("   model = mlflow.spark.load_model(model_uri)")
    print("   predictions = model.transform(feature_data)")
    
else:
    print("❌ Model Training: FAILED")
    print("❌ Performance Evaluation: SKIPPED")
    print("❌ Model Registration: SKIPPED")
    print()
    print("🔧 TROUBLESHOOTING STEPS:")
    print("1. 📊 Verify feature tables exist (run feature engineering)")
    print("2. 🔌 Check Spark and MLflow connectivity")
    print("3. 🏗️  Verify Unity Catalog permissions")
    print("4. 📝 Review error messages above")
    print("5. 🔄 Re-run notebook after resolving issues")

print()
print("📚 ADDITIONAL RESOURCES:")
print("- Model monitoring: Set up drift detection")
print("- A/B testing: Compare with existing models")
print("- Feature engineering: Continuously improve features")
print("- Performance tracking: Monitor real-world accuracy")

print(f"\n⏰ Notebook completed at: {datetime.now()}")